In [7]:
import pandas as pd
df = pd.read_csv("./OPEN_MEDIC_2022.CSV",sep=";",encoding='latin-1')
df.head(10)



,ATC1,l_ATC1,ATC2,L_ATC2,ATC3,L_ATC3,ATC4,L_ATC4,ATC5,L_ATC5,...,l_cip13,TOP_GEN,GEN_NUM,age,sexe,BEN_REG,PSP_SPE,BOITES,REM,BSE
0,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,5,99,17,"8,09","28,69"
1,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,11,99,24,"9,29","30,96"
2,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,44,99,20,"8,39","25,80"
3,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,84,99,13,"5,03","16,77"
4,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,99,1,14,"5,42","18,06"
5,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,99,12,15,"5,81","19,35"
6,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,1,99,99,11,"5,16","14,19"
7,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,2,5,99,16,"7,57","26,94"
8,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,2,11,99,21,"7,74","27,09"
9,A,SYSTEME DIGESTIF ET METABOLISME,A01,PREPARATIONS STOMATOLOGIQUES,A01A,PREPARATIONS STOMATOLOGIQUES,A01AA,MEDICAMENTS PROPHYLACTIQUES ANTICARIES,A01AA01,SODIUM FLUORURE,...,"ZYMAFLUOR 0,25MG CPR 200",0,0,0,2,44,99,23,"8,51","29,67"


In [17]:
atc_lib = pd.read_csv("./ATC5.csv",sep=";",encoding='latin-1')
region_lib = pd.read_csv("./region.csv",sep=";",encoding='latin-1')
sexe_lib = pd.read_csv("./sexe.csv",sep=";",encoding='latin-1')
age_lib = pd.read_csv("./age.csv",sep=";",encoding='latin-1')

,atc,lib_atc
0,A01AA01,SODIUM FLUORURE
1,A01AA02,SODIUM MONOFLUOROPHOSPHATE
2,A01AA51,SODIUM FLUORURE EN ASSOCIATION
3,A01AB03,CHLORHEXIDINE
4,A01AB09,MICONAZOLE
